In [1]:
import nltk # 자연어 툴킷
import threading # 스레딩 기반 병렬 처리
import queue # 대기열 라이브러리
import feedparser # 구문 분석(파싱) 라이브러리
import uuid # uuid 버전 1, 3, 4, 5 생성 라이브러리

In [2]:
threads = [] # 프로그램의 모든 스레드 추적
queues = [queue.Queue(), queue.Queue()] # 변수 queue에 두 개의 대기열 목록 생성(토큰화된 문장 저장, 분석된 모든 품사 단어 저장)

In [3]:
# 샘플 RSS 피드를 읽고 단어와 텍스트의 고유 식별자 함께 저장
def extractWords():
    url = 'https://timesofindia.indiatimes.com/rssfeeds/1081479906.cms'
    feed = feedparser.parse(url) # URL 내용 다운로드 후 뉴스항목(제목과 요약 키가 있는 딕셔너리) 리스트로 변환
    for entry in feed['entries'][:5]: # RSS 피드의 5개 항목을 가져와 반복
        text = entry['title'] # 현재 RSS 피드의 제목이 text로 할당
        if 'ex' in text: # 제목에 'ex'가 포함되어 있으면 건너뛰기
            continue

        words = nltk.word_tokenize(text) # 입력 테스트를 단어로 분리
        data = {'uuid' : uuid.uuid4(), 'input' : words} # uuid 및 입력 단어 저장
        queues[0].put(data, True) # 딕셔너리를 queues[0]에 저장, 두 번째 인수는 대기열이 가득 찬 경우 일시 중지
        print('>> {} : {}'.format(data['uuid'], text)) # 고유 ID와 처리 중인 RSS 항목 출력

In [4]:
def extractPOS():
    while True:
        if queues[0].empty(): # 첫 번째 대기열에 데이터가 없을 경우
            break
        else: # 첫 번째 대기열에 데이터가 있을 경우
            data = queues[0].get() # 대기열에서 첫 번째 항목 가져오기
            words = data['input'] 
            postags = nltk.pos_tag(words) # 품사 태깅 : 첫 번째 요소는 원래 단어, 두 번째 요소는 품사
            queues[0].task_done() # 첫 번째 대기열을 처리 완료로 업데이트
            queues[1].put({'uuid' : data['uuid'], 'input' : postags}, True) # 품사 태깅된 단어 목록을 두 번째 대기열에 저장

In [5]:
def extractNE(): # 단어에서 개체명 추출
    while True:
        if queues[1].empty(): # 두 번째 대기열에 데이터가 없을 경우
            break
        else: # 두 번째 대기열에 데이터가 있을 경우
            data = queues[1].get() # 두 번째 대기열에서 데이터 가져오기
            postags = data['input'] # 항목을 가져와서 저장
            queues[1].task_done() # 방금 선택한 항목에 대해 처리 완료 표시
            
            chunks = nltk.ne_chunk(postags, binary = False) # 개체명 태깅 수행
            print(' << {} : '.format(data['uuid']), end = '') # 현재 UUID 출력
            for path in chunks: # chunks를 하나씩 추출
                try: # 개체명이 태깅된 경우
                    label = path.label()  
                    print(path, end = ', ') # 개체명이 태그된 단어 출력
                except:
                    pass
            print()

In [6]:
def runProgram(): # 스레드를 이용해 파이프라인 구성
    # extractWords 함수를 갖는 스레드 생성 후 시작, threads 리스트에 추가
    e = threading.Thread(target = extractWords()) 
    e.start()
    threads.append(e)
    
    # extractPOS 함수를 갖는 스레드 생성 후 시작, threads 리스트에 추가
    p = threading.Thread(target = extractPOS())
    p.start()
    threads.append(p)
    
    # extractNE 함수를 갖는 스레드 생성 후 시작, threads 리스트에 추가
    n = threading.Thread(target= extractNE())
    n.start()
    threads.append(n)
    
    # 모든 처리가 완료되면 할당된 자원 해제
    queues[0].join()
    queues[1].join()
    
    # thread에 있는 모든 항목 반복 후 할당된 자원 해제
    for t in threads:
        t.join()

In [7]:
if __name__ == '__main__': # 메인 스레드와 함께 실행
    runProgram()

>> 2b9f18bd-010a-471a-9579-a7776340de45 : 'Was not taken seriously in South film industry'
>> 4d789d40-9c23-4a92-b29b-bcb624881bf2 : SRK, Salman to shoot for Tiger 3 in June
>> d6ff2cc8-3d11-41a6-8e35-6736c505c19c : RRR box office collection day 2
>> 7397010b-29b7-434f-aaad-054ed295c58c : Similar series that screened at the same time
>> bc0d76de-6944-4f48-a828-35154e8faf61 : Anupam: Salman congratulated for Kashmir Files
 << 2b9f18bd-010a-471a-9579-a7776340de45 : (GPE South/NNP), 
 << 4d789d40-9c23-4a92-b29b-bcb624881bf2 : (PERSON Salman/NNP), 
 << d6ff2cc8-3d11-41a6-8e35-6736c505c19c : 
 << 7397010b-29b7-434f-aaad-054ed295c58c : (GPE Similar/JJ), 
 << bc0d76de-6944-4f48-a828-35154e8faf61 : (GPE Anupam/NN), (PERSON Salman/NNP), (PERSON Kashmir/NNP Files/NNP), 
